In [17]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import warnings
import re
warnings.filterwarnings('ignore')
import pandas as pd
import sklearn
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
harmful_2_df = pd.read_csv('harmful_7_learning.csv')


harmful_url_dic = { 'blrblr':0,
                   
                   'sex': 1, 'porn': 1, 'gay': 1, 'tube': 1, 'pornhub':1,
                   'xxx': 1, 'fuck': 1, 'dick': 1, 'tit': 2,
                   'anal': 1, 'asian': 1, 'adult': 1, 'shemal': 1,'lesbian': 1,

                   'teen': 2, 'big': 2, 'girl': 2, 'pussi': 2, 'ass': 2, 'amateur': 2, 'matur': 2,
                   'cock': 2,'milf': 2, 'sexi': 2, 'babe':2, 'nud':2, 'nude': 2,
                   'blond': 2, 'blowjob': 2, 'cum': 2, 'young': 2,
                   'hardcor': 2, 'hardcore': 2 ,

                   'game': 3, 'casino': 3, 
                   'card': 3,  'race': 3,  

                   'escort': 4, 'servic': 4, 'service':4, 'agenc': 4,
                   'galleri': 4,'massag': 4, 'model': 4,'vip': 4, 'profil': 4,
                   
                   
                   'cam':5, 'video':5, 'chat':5, 'webcam': 5,
                   
                   'domain':7, 'domains':7, 'gateway':7
}

def word_idx(top20):
    list = top20.split()
    for word in list:
        if word in harmful_url_dic:
            return harmful_url_dic[word]
            break;
    return 0

def img_idx(num):
    if num < 10 : return 0
    elif (num >=10)&(num<50): return 1
    elif (num >=50)&(num<100): return 2
    elif (num >=100)&(num<200): return 3
    else: return 4

def child_idx(num):
    if num < 5 : return 0
    elif (num >=5)&(num<10): return 1
    elif (num >=10)&(num<25): return 2
    else: return 3

def img_child(img, child):
    if img >= child: return 0
    else: return 1
    
def url_idx(idx):
    if idx == 0: return 0
    else: return 1
    
def target(wi, ic, hw):
    if hw < 0.125 :
        if wi == 0: return 0
        elif wi == 7: return 7
        else: return wi
        
    elif hw >= 0.125 and hw < 0.5:
        if ic == 1 : return 6
        else: return wi
        
    # harmful_weight >= 0.5    
    else:
        if wi == 0 or wi ==7 : return 7
        elif ic == 1: return 6
        else: return wi
harmful_2_df['top_word'] = harmful_2_df['top_word'].fillna(" ")
harmful_2_df['top_word'] = harmful_2_df['top_word'].apply(lambda x : re.sub("[^a-zA-Z]"," ", x))

In [18]:
import math, sys
# from konlpy.tag import Twitter


class BayesianFilter:
    """ 베이지안 필터 """
    def __init__(self):
        self.words = set() # 출현한 단어 기록
        self.word_dict = {} # 카테고리마다의 출현 횟수 기록
        self.category_dict = {} # 카테고리 출현 횟수 기록
    # 형태소 분석하기 --- (※1)
    def split(self, text):
        results = []
        list = text.split()
        for word in list:
            results.append(word)
        return results
    # 단어와 카테고리의 출현 횟수 세기 --- (※2)
    def inc_word(self, word, category):
        # 단어를 카테고리에 추가하기
        if not category in self.word_dict:
            self.word_dict[category] = {}
        if not word in self.word_dict[category]:
            self.word_dict[category][word] = 0
        self.word_dict[category][word] += 1
        self.words.add(word)
    def inc_category(self, category):
        # 카테고리 계산하기
        if not category in self.category_dict:
            self.category_dict[category] = 0
        self.category_dict[category] += 1
    
    # 텍스트 학습하기 --- (※3)
    def fit(self, text, category):
        """ 텍스트 학습 """
        word_list = self.split(text)
        for word in word_list:
            self.inc_word(word, category)
        self.inc_category(category)
    
    # 단어 리스트에 점수 매기기--- (※4)
    def score(self, words, category):
        score = math.log(self.category_prob(category))
        for word in words:
            score += math.log(self.word_prob(word, category))
        return score
    
    # 예측하기 --- (※5)
    def predict(self, text):
        best_category = None
        max_score = -sys.maxsize 
        words = self.split(text)
        score_list = []
        for word in words:
            if word in harmful_url_dic:
                for category in self.category_dict.keys():
                    score = self.score(words, category)
                    score_list.append((category, score))
                    if score > max_score:
                        max_score = score
                        best_category = category
            else:
                for category in self.category_dict.keys():
                    score = self.score(words, 0)
                    score_list.append((category, 0))
                    if score > max_score:
                        max_score = score
                        best_category = category

        return best_category
    # 카테고리 내부의 단어 출현 횟수 구하기
    def get_word_count(self, word, category):
        if word in self.word_dict[category]:
            return self.word_dict[category][word]
        else:
            return 0
    # 카테고리 계산
    def category_prob(self, category):
        sum_categories = sum(self.category_dict.values())
        category_v = self.category_dict[category]
        return category_v / sum_categories
        
    # 카테고리 내부의 단어 출현 비율 계산 --- (※6)
    def word_prob(self, word, category):
        n = self.get_word_count(word, category) + 1 # ---(※6a)
        d = sum(self.word_dict[category].values()) + len(self.words)
        return n / d


bf = BayesianFilter()
for i in harmful_url_dic:
    bf.fit(i, harmful_url_dic[i])



In [19]:
print(harmful_2_df.shape)
harmful_2_df.head(3)

(297, 9)


,url_id,url,url_harmful_idx,harmful_word_num,harmful_child_num,img_num,child_num,harmful_weight,top_word
0,1755,tumblr.com,0,2,0,5,2,0.050,blog tumblr peopl log follow like link magic m...
1,1883,youtube.com,0,1,0,79,5,0.025,play durat nextplay ago music views year rela...
2,1893,google.com,0,2,0,1,3,0.050,advanc drive gmail googl googlegoogle co uk h...


In [20]:
harmful_2_df['word_idx'] = harmful_2_df['top_word'].apply(lambda x : bf.predict(x))
harmful_2_df['url_harmful_idx'] = harmful_2_df['url_harmful_idx'].apply(lambda x : url_idx(x))
harmful_2_df['img_child'] = harmful_2_df.apply(lambda x : img_child(x['img_num'],x['child_num']),axis=1)
harmful_2_df['img_num'] = harmful_2_df['img_num'].apply(lambda x : img_idx(x))
harmful_2_df['child_num'] = harmful_2_df['child_num'].apply(lambda x : child_idx(x))
 
harmful_2_df['target'] = harmful_2_df.apply(lambda x : target(x['word_idx'],x['img_child'],x['harmful_weight']),axis=1)

harmful_2_df.head(3)

,url_id,url,url_harmful_idx,harmful_word_num,harmful_child_num,img_num,child_num,harmful_weight,top_word,word_idx,img_child,target
0,1755,tumblr.com,0,2,0,0,0,0.050,blog tumblr peopl log follow like link magic m...,0,0,0
1,1883,youtube.com,0,1,0,2,1,0.025,play durat nextplay ago music views year rela...,0,0,0
2,1893,google.com,0,2,0,0,0,0.050,advanc drive gmail googl googlegoogle co uk h...,0,1,0


In [21]:
value_counts=harmful_2_df['target'].value_counts()
print(value_counts)

0    223
7     56
3      7
6      5
5      4
4      2
Name: target, dtype: int64


In [22]:
view_df = harmful_2_df[['url_id','url','harmful_weight','target']]

In [33]:
view_df.loc[view_df['target'] == 4 ]

,url_id,url,harmful_weight,target
124,264651,mediagoogle.net,0.025,4
183,357386,heraldedu.co.kr,0.050,4
